In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
plt.style.use('ggplot')
import datetime as dt
import seaborn as sns
%matplotlib inline

In [3]:
trend = pd.read_csv('trend.csv',parse_dates=['date'])
route = pd.read_csv('route.csv',parse_dates=['date'])
patient = pd.read_csv('patient.csv',parse_dates=['confirmed_date'])
time=pd.read_csv('time.csv',parse_dates=['date'])

FileNotFoundError: [Errno 2] File trend.csv does not exist: 'trend.csv'

# Visualization - 1 (EDA)

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(time['date'],time['new_confirmed'],'ro:')
# sns.lineplot(data=time,x='date',y='new_confirmed',markers=True,dashes=True)
plt.xticks(rotation=30)
plt.title('How many COVID-19 confirmed peple')

In [ ]:
fig,ax1=plt.subplots(figsize=(12,8))
ax2 = ax1.twinx()
confirmed_pivot = time.pivot_table(index='date',values='new_confirmed',aggfunc='sum').reset_index()
sns.lineplot(x=confirmed_pivot['date'],y=confirmed_pivot['new_confirmed'].cumsum(),ax=ax1,label='confirmed')
ax1.set_ylim=[0,10000]
ax1.set_ylabel('Confirmation Count')
ax1.legend(loc='upper left')
ax1.tick_params(rotation=60)

deceased_pivot = time.pivot_table(index='date',values='new_deceased',aggfunc='sum').reset_index()
sns.lineplot(x=deceased_pivot['date'],y=deceased_pivot['new_deceased'].cumsum(),color='blue',ax=ax2,label='deceased')
ax2.set_ylabel('Decease Count')
ax2.set_ylim=[0,100]
ax2.legend(loc='upper center')
ax2.tick_params(rotation=60)


plt.title('Confirmation and Decease Accumulate Count')
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
released_pivot = time.pivot_table(index='date',values='new_released',aggfunc='sum').reset_index()
sns.lineplot(x=released_pivot['date'],y=released_pivot['new_released'].cumsum(),label='released')
deceased_pivot = time.pivot_table(index='date',values='new_deceased',aggfunc='sum').reset_index()
sns.lineplot(x=deceased_pivot['date'],y=deceased_pivot['new_deceased'].cumsum(),color='blue',label='deceased')
plt.xticks(rotation=30)
plt.legend(loc=0)
plt.title('Released and Deceased accumulate Count')

In [ ]:
fig, (ax1,ax2) = plt.subplots(ncols=2,figsize=(12,8))
sns.countplot(patient['group'],ax=ax1)
ax1.tick_params(labelrotation=90)
ax1.set(title='Belonging to the infected')
sns.countplot(patient['infection_reason'],ax=ax2)
ax2.tick_params(labelrotation=90)
ax2.set(title='Infection Reason')

신천지 교회의 방문이 코로나 확산에 많은 기여를 했으므로 group이 신천지 교회인 사람들 중 감염 이유가 결측치인 사람들을 신천지 교회 방문으로 결측치를 대체하겠다.

Since the visit of Shincheonji Church has contributed a lot to the spread of corona, we will replace the missing ones by visiting the Shincheonji church among those whose group is Shincheonji Church.

In [ ]:
idx= patient[patient['group'] == 'Shincheonji Church']['infection_reason'].isnull().index

In [ ]:
patient['infection_reason'][idx] = 'visit to Shincheonji Church'

In [ ]:
sns.countplot(patient['infection_reason'])
plt.xticks(rotation=90)

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(patient['region'])
plt.xticks(rotation=90)
plt.title('Where is most infected')

In [ ]:
import folium
southkorea_map = folium.Map(location=[36.55,126.983333 ], zoom_start=7,tiles='Stamen Toner')

for lat, lon,city in zip(route['latitude'], route['longitude'],route['city']):
    folium.CircleMarker([lat, lon],
                        radius=5,
                        color='red',
                      popup =('City: ' + str(city) + '<br>'),
                        fill_color='red',
                        fill_opacity=0.7 ).add_to(southkorea_map)
southkorea_map

수도권에 사는 사람이 가장 많이 확진이 되었고 경상북도,대구가 뒤를 이었다.

People living in the capital area were the most confirmed, followed by Gyeongsangbuk-do and Daegu

In [ ]:
import plotly.express as px
states = pd.DataFrame(patient["state"].value_counts())
states["status"] = states.index
states.rename(columns={"state": "count"}, inplace=True)

fig = px.pie(states,
             values="count",
             names="status",
             title="Current state of patients",
             template="seaborn")
fig.update_traces(rotation=90, pull=0.05, textinfo="value+percent+label")
fig.show()
# Thanks for Vansh Jatana

# feature Enginerring

In [ ]:
col = ["confirmed_date", "released_date", "deceased_date"]
for i in col:
    patient[i] = pd.to_datetime(patient[i])
patient['country'].fillna(patient['country'].mode()[0],inplace=True)
patient['confirmed_date'].fillna(dt.datetime(2020,2,2),inplace=True)
patient['state'].fillna(patient['state'].mode()[0],inplace=True)

In [ ]:
for i in range(patient.shape[0]):
    if type(patient['deceased_date'][i]) == float:
        if patient['state'][i] == 'released':
            patient['deceased_date'][i] = 'survived'
        elif patient['state'][i] == 'isolated':
            patient['deceased_date'][i] = "don't know"
    else:
        continue
for i in range(patient.shape[0]):
    if type(patient['released_date'][i]) == float:
        if patient['state'][i] == 'deceased':
            patient['released_date'][i] = 'deceased'
        else:
            patient['released_date'][i] = "don't know"
    else:
        continue

In [ ]:
idx = patient[patient['group']=='Shincheonji Church'].index
patient['infected_by'][idx] = 31.0
patient['group'].fillna('None',inplace=True)
patient['infection_order'] = patient['infection_order'].fillna(0.0).astype(int)

In [ ]:
patient['birth_year'] = patient['birth_year'].fillna(0.0).astype(int)
patient['birth_year'] = patient['birth_year'].apply(lambda x: x if x>0 else np.nan)
patient['age'] = 2020-patient['birth_year'] + 1 # korean age
import math
def age_range(age):
    if age >= 0: # not NaN
        if age % 10 != 0:
            lower = int(math.floor(age / 10.0)) * 10
            upper = int(math.ceil(age / 10.0)) * 10 - 1
            return f"{lower}-{upper}"
        else:
            lower = int(age)
            upper = int(age + 9) 
            return f"{lower}-{upper}"
    return "Unknown"
patient['age_range'] = patient['age'].apply(lambda x: age_range(x))

In [ ]:
patient['relased_time'] = patient['released_date'] - patient['confirmed_date']
patient['deceased_time'] = patient['deceased_date'] - patient['confirmed_date']

In [ ]:
tmp=[]
idx = []
for i in range(patient.shape[0]):
    if type(patient['sex'][i]) == str:
        tmp.append(patient['age_range'][i]+'_'+patient['sex'][i])
        idx.append(i)
    else:
        continue
patient['age_sex']='None'
patient['age_sex'][idx] = tmp

In [ ]:
patient.tail()

# Visualization - 2

In [ ]:
released = patient[patient['state'] == 'released']
fig,(ax1,ax2) = plt.subplots(ncols=2,figsize=(20,5))
sns.countplot(released['age_range'].sort_values(),ax=ax1)
sns.kdeplot(released['age'],shade=True,ax=ax2)

50대의 완치자 수 가 가장 많다.


50s had the highest number of released people

In [ ]:
tmp = pd.DataFrame(released['age_range'].value_counts()/patient['age_range'].value_counts())
tmp = tmp.dropna()
tmp.plot(kind='bar')

전체 확진자 수 대비 완치자 수를 비교한 결과 역시 50대가 가장 많았다. 특이한 점은 10대의 비율이 급격하게 상승했다는 것이다.

Compared to the total number of confirmed patients, 50s was the most frequent result. 
The peculiarity is that the proportion of teenagers has risen sharply.

In [ ]:
deceased = patient[patient['state'] == 'deceased']
fig,(ax1,ax2) = plt.subplots(ncols=2,figsize=(20,5))
sns.countplot(deceased['age_range'].sort_values(),ax=ax1)
ax1.set(title='Deceased')
sns.kdeplot(deceased['age'],shade=True,ax=ax2)

In [ ]:
tmp = pd.DataFrame(deceased['age_range'].value_counts()/patient['age_range'].value_counts())
tmp = tmp.dropna()
tmp.plot(kind='bar')

나이가 많을수록 사망 확률이 높다는 것을 알 수 있다.


The older the confirmed person, the more likely it is to die.


In [ ]:
fig,(ax1,ax2,ax3) = plt.subplots(ncols=3,figsize=(20,8))
sns.countplot(deceased['age_sex'].sort_values(ascending=True),ax=ax1)
ax1.tick_params(rotation=90)
sns.countplot(deceased['age_range'],hue=deceased['sex'],ax=ax2)
sns.countplot(deceased['sex'],ax=ax3)

즉, 나이가 많고 남자일 수록 사망할 확률이 높다.


In other words, older and older men are more likely to die.

In [ ]:
sns.countplot(deceased['region'])

사망자의 대부분은 대구나 경상북도 사람이다.


Most of the dead live in Daegu or Gyeongsangbuk-do

정리하자면 대구나 경상북도에 거주하는 고령의 남성이 가장 사망할 확률이 높다고 할 수 있다.

In summary, older men living in Daegu or Gyeongsangbuk-do are most likely to die.

# Modeling

In [ ]:
daily_count = patient.groupby(patient.confirmed_date).id.count()
dataset = daily_count.resample('D').first().fillna(0).cumsum()
dataset = dataset[20:]

In [ ]:
train_x = np.arange(len(dataset)).reshape(-1, 1)
train_y = dataset.values
test_x = np.arange(len(dataset)+7).reshape(-1,1)

In [ ]:
from fbprophet import Prophet

In [ ]:
prophet = pd.DataFrame(dataset).reset_index()
prophet.columns = ['ds','y']
prophet

In [ ]:
m=Prophet()
m.fit(prophet)

In [ ]:
future = m.make_future_dataframe(periods=10)
forecast = m.predict(future)
forecast

In [ ]:
from fbprophet.plot import plot_plotly
import plotly.offline as py
fig = plot_plotly(m, forecast)
py.iplot(fig) 
fig = m.plot(forecast,xlabel='Date',ylabel='Confirmed Count')